# Data Gathering

In [1]:
import pandas as pd
import json
from timeit import default_timer as timer
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### 1. Create a DF out of the desired data from the JSON file created using the Twitter API

In [2]:
tweet_data = []
#Open and read each line of the Json file. 
with open("tweet_json.txt", 'r') as json_file:
    for line in json_file:
        json_data = json.loads(line)
        timestamp = json_data['created_at'] #Pull out 4 data points (timestamp, tweet_id, likes, and retweets)
        likes = json_data['favorite_count']
        tweet_id = json_data['id']
        retweets = json_data['retweet_count']
        
        tweet_data.append({'timestamp': timestamp, #Append those 4 pieces of data to a list in the form of a dict (key/value pairs) to be converted later into a DF
                         'likes': likes,
                         'tweet_id': tweet_id,
                         'retweets': retweets})


In [3]:
tweet_data_df = pd.DataFrame(tweet_data, columns = ['timestamp', 'likes', 'tweet_id', 'retweets'])#Create the dataframe with the JSON data above and specific column names

In [4]:
tweet_data_df.head()

,timestamp,likes,tweet_id,retweets
0,Tue Aug 01 16:23:56 +0000 2017,35562,892420643555336193,7534
1,Tue Aug 01 00:17:27 +0000 2017,30753,892177421306343426,5589
2,Mon Jul 31 00:18:03 +0000 2017,23130,891815181378084864,3703
3,Sun Jul 30 15:58:51 +0000 2017,38869,891689557279858688,7717
4,Sat Jul 29 16:00:24 +0000 2017,37129,891327558926688256,8316


In [5]:
tweet_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  2331 non-null   object
 1   likes      2331 non-null   int64 
 2   tweet_id   2331 non-null   int64 
 3   retweets   2331 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 73.0+ KB


### 2. Create a DF from the provided WeRateDogs twitter archive data csv

In [6]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [7]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [8]:
twitter_archive.in_reply_to_status_id.isnull().value_counts()

True     2278
False      78
Name: in_reply_to_status_id, dtype: int64

### 3. Create a DF utilizing the requests package to work with the provided image predictions url

In [9]:
import requests
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

In [10]:
r = requests.get(url)

In [11]:
with open('image_predictions.tsv', 'wb') as file:
    file.write(r.content)

In [12]:
image_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [13]:
image_predictions.sample(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
285,671147085991960577,https://pbs.twimg.com/media/CVBktzQXAAAPpUA.jpg,1,Yorkshire_terrier,0.467202,True,cairn,0.440122,True,silky_terrier,0.058690,True
1966,867774946302451713,https://pbs.twimg.com/media/DAr0tDZXUAEMvdu.jpg,2,Border_collie,0.661953,True,Cardigan,0.175718,True,collie,0.087142,True
1106,723179728551723008,https://pbs.twimg.com/media/CglAHjAUgAAfxcq.jpg,1,tennis_ball,0.176495,False,badger,0.059905,False,Norwegian_elkhound,0.056850,True


#### Now that the DFs have been created, a copy of each is generated for future use during assessment or cleaning stages of the Data Wrangling process

In [14]:
tweet_data_df_clean = tweet_data_df.copy()
image_predictions_clean = image_predictions.copy()
twitter_archive_clean = twitter_archive.copy()

# Assess

### 1. Programmatic and Visual Assessment

#### Quality
<ul> 
    <li>tweet_data_df_clean:
        <ol>
            <li>Timestamp data provides more data than necessary</li>
            <li>There are 163 tweets with 0 likes, but all of them have at least 19 or more retweets</li>
        </ol>
    </li>
    <li>image_predictions_clean
    </li>
    <li>twitter_archive_clean
    </li>
</ul>        

#### Tidiness
<ul> 
    <li>tweet_data_df_clean:
        <ol>
            <li>timestamp is datatype is an object and it should be datetime</li>
            <li>tweet_id datatype is int64 and it should be a string</li>
        </ol>
    </li>
    <li>image_predictions_clean
    </li>
    <li>twitter_archive_clean
    </li>
</ul>        

## tweet_data_df_clean

In [48]:
tweet_data_df_clean.sort_values(by = 'retweets').head(10)

,timestamp,likes,tweet_id,retweets
279,Sat Mar 04 17:56:49 +0000 2017,138,838085839343206401,1
1271,Thu Mar 10 17:35:20 +0000 2016,49,707983188426153984,2
262,Sat Mar 11 22:59:09 +0000 2017,171,840698636975636481,2
328,Thu Feb 16 04:45:50 +0000 2017,62,832088576586297345,2
109,Fri Jun 02 19:38:25 +0000 2017,113,870726314365509632,3
29,Sat Jul 15 16:51:35 +0000 2017,110,886267009285017600,4
1056,Sat Jun 04 00:32:32 +0000 2016,106,738891149612572673,6
54,Sun Jul 02 21:58:53 +0000 2017,115,881633300179243008,7
411,Tue Jan 17 00:33:26 +0000 2017,254,821153421864615936,10
63,Tue Jun 27 12:14:36 +0000 2017,291,879674319642796034,10


In [16]:
tweet_data_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  2331 non-null   object
 1   likes      2331 non-null   int64 
 2   tweet_id   2331 non-null   int64 
 3   retweets   2331 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 73.0+ KB


In [17]:
tweet_data_df_clean.describe()

,likes,tweet_id,retweets
count,2331.00000,2.331000e+03,2331.000000
mean,7430.58387,7.419079e+17,2648.574432
std,11539.21545,6.823170e+16,4479.174285
min,0.00000,6.660209e+17,1.000000
25%,1292.50000,6.782670e+17,537.000000
50%,3229.00000,7.182469e+17,1239.000000
75%,9097.00000,7.986692e+17,3077.000000
max,153472.00000,8.924206e+17,76095.000000


In [27]:
tweet_data_df_clean.likes.sort_values()

453          0
439          0
437          0
434          0
431          0
         ...  
1055    113395
129     114540
517     118456
397     130823
1015    153472
Name: likes, Length: 2331, dtype: int64

In [37]:
tweet_data_df_clean.likes.value_counts().sort_values().tail(11)

110       2
3062      3
1362      3
2771      3
752       3
491       3
291       3
755       3
2033      3
3727      3
0       163
Name: likes, dtype: int64

In [24]:
tweet_data_df_clean.query("likes == 0")

,timestamp,likes,tweet_id,retweets
31,Sat Jul 15 02:45:48 +0000 2017,0,886054160059072513,99
35,Thu Jul 13 01:35:06 +0000 2017,0,885311592912609280,16528
67,Mon Jun 26 00:13:58 +0000 2017,0,879130579576475649,6094
72,Sat Jun 24 00:09:53 +0000 2017,0,878404777348136964,1148
73,Fri Jun 23 18:17:33 +0000 2017,0,878316110768087041,5950
...,...,...,...,...
999,Sat Jun 25 01:52:36 +0000 2016,0,746521445350707200,966
1019,Fri Jun 17 16:01:16 +0000 2016,0,743835915802583040,2024
1218,Mon Mar 21 19:31:59 +0000 2016,0,711998809858043904,125
2234,Fri Nov 20 03:51:52 +0000 2015,0,667550904950915073,31


In [25]:
tweet_data_df_clean.query("retweets == 1")

,timestamp,likes,tweet_id,retweets
279,Sat Mar 04 17:56:49 +0000 2017,138,838085839343206401,1


In [26]:
tweet_data_df_clean.query("likes == 0 and retweets <= 20")

,timestamp,likes,tweet_id,retweets
203,Tue Apr 11 18:15:55 +0000 2017,0,851861385021730816,19


In [40]:
tweet_data_df_clean.retweets.sort_values()

279         1
1271        2
262         2
328         2
109         3
        ...  
65      39888
397     42889
517     55468
1055    56601
1015    76095
Name: retweets, Length: 2331, dtype: int64

## image_predictions_clean